### Задача 3

Вместо того, чтобы заталкивать в тест «поюзерные» CTR, можно сконструировать другую метрику и анализировать ее, но при этом гарантируется (в отличие от сглаженного CTR), что если тест на этой другой метрике «прокрасится» и увидит изменения, значит изменения есть и в метрике исходной (то есть в лайках на пользователя и в пользовательских CTR.

Считаем общий CTR в контрольной группе  

- 𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙=𝑠𝑢𝑚(𝑙𝑖𝑘𝑒𝑠)/𝑠𝑢𝑚(𝑣𝑖𝑒𝑤𝑠) 

- Посчитаем в обеих группах поюзерную метрику  𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠=𝑙𝑖𝑘𝑒𝑠−𝐶𝑇𝑅𝑐𝑜𝑛𝑡𝑟𝑜𝑙∗𝑣𝑖𝑒𝑤𝑠 

- После чего сравним  t-тестом отличия в группах по метрике 𝑙𝑖𝑛𝑒𝑎𝑟𝑖𝑧𝑒𝑑_𝑙𝑖𝑘𝑒𝑠  


In [1]:
import pandahouse
import swifter
import hashlib
import seaborn as sns
import numpy as np
from scipy import stats
import pandas as pd

In [39]:
#выгружаем данные из clickhouse
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user':'student',
    'database':'simulator_20220620'
}

q = """
select exp_group, user_id, 
       sum(action = 'like') as likes,
       sum(action = 'view') as views,
       likes/views as ctr
from {db}.feed_actions
where toDate(time) >= '2022-05-24' and toDate(time) <= '2022-05-30'
      and exp_group in (0, 1, 2, 3)
group by exp_group, user_id
"""

df = pandahouse.read_clickhouse(q, connection = connection)

df.head(5)

,exp_group,user_id,likes,views,ctr
0,3,115383,9,30,0.300000
1,1,18392,7,32,0.218750
2,2,131473,14,134,0.104478
3,3,123580,13,48,0.270833
4,2,32420,26,128,0.203125


In [40]:
#расчет CTR в контрольных группах и linearized_likes в группах
#0, 1 - контрольные группы, новый алгоритм раскатили на 2, 3 группы

#0 группа контрольная, 3 группа тестовая
CTRcontrol_0 = df[df.exp_group == 0].likes.sum()/df[df.exp_group == 0].views.sum()
linearized_likes_0 = df[df.exp_group == 0].likes - CTRcontrol_0*df[df.exp_group == 0].views
linearized_likes_3 = df[df.exp_group == 3].likes - CTRcontrol_0*df[df.exp_group == 3].views

#1 группа контрольная, 2 группа тестовая
CTRcontrol_1 = df[df.exp_group == 1].likes.sum()/df[df.exp_group == 1].views.sum()
linearized_likes_1 = df[df.exp_group == 1].likes - CTRcontrol_1*df[df.exp_group == 1].views
linearized_likes_2 = df[df.exp_group == 2].likes - CTRcontrol_1*df[df.exp_group == 2].views

Анализ теста между группами 0 и 3 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [41]:
stats.ttest_ind(linearized_likes_0, linearized_likes_3, equal_var = False)

Ttest_indResult(statistic=-15.214995460903827, pvalue=5.4914249479690016e-52)

In [42]:
stats.mannwhitneyu(linearized_likes_0, linearized_likes_3, alternative = 'two-sided')

MannwhitneyuResult(statistic=43338326.0, pvalue=1.1517315029786597e-63)

#### p value стремится к нулю, а значит между группами 0 и 3 есть отличие.

Анализ теста между группами 1 и 2 по метрике линеаризованных лайков. Видно ли отличие? Стало ли 𝑝−𝑣𝑎𝑙𝑢𝑒 меньше?

In [43]:
stats.ttest_ind(linearized_likes_1, linearized_likes_2, equal_var = False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

In [44]:
stats.mannwhitneyu(linearized_likes_1, linearized_likes_2, alternative = 'two-sided')

MannwhitneyuResult(statistic=54534537.0, pvalue=9.404319579641738e-27)

#### p value стремится к нулю, а значит между группами 1 и 2 есть отличие.